In [ ]:
# 코랩 사용하기 전 실행하는 절차
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)


In [ ]:
# 코랩 사용 2단계
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
# 데이터 전처리 1단계 - 데이터 확인
data.info()
data.describe()
data.isna().sum()

In [ ]:
# 데이터 전처리 2단계 - 데이터 변경
data['v1'] = data['v1'].replace(['ham','spam'],[0,1]) 
data.drop_duplicates(subset=['v2'], inplace=True)         # v2를 기준하여 중복되는 요소 제거

In [ ]:
# 데이터 전처리 3단계 - 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

In [ ]:
# 기타. 확인기
print('v2열의 유니크한 값 :',data['v2'].nunique())
print('메일의 최대 길이 : %d' % max(len(sample) for sample in X_train_encoded))
print('메일의 평균 길이 : %f' % (sum(map(len, X_train_encoded))/len(X_train_encoded)))
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))

In [ ]:
# 시드 고정하는 함수
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
# GPT STT 시스템
import openai
# 키 입력 이후 소리를 텍스트로 구현 가능
def STT(audio):                                                                # 소리를 텍스트로 구현하는 함수
    # 파일 저장
    filename='input.mp3'                                                       # 파일이름은 같은위치의 인풋
    wav_file = open(filename, "wb")                                            # 파일을 열기.
    wav_file.write(audio.tobytes())                                            # 파일을 글자로 적기
    wav_file.close()                                                           # 작성하고 저장.

    # 음원 파일 열기
    audio_file = open(filename, "rb")                                          # 오디오파일 열기
    #Whisper 모델을 활용해 텍스트 얻기                             
    transcript = openai.Audio.transcribe("whisper-1", audio_file)              # 오픈AI 위스퍼로 텍스트 얻기
    audio_file.close()                                                         # 오디오파일 닫기
    # 파일 삭제
    os.remove(filename)                                                        # 파일 삭제하기
    return transcript["text"]                                                  # 텍스트 리턴하기 

In [ ]:
# GPT에게 질문하기
import openai

def ask_gpt(prompt, model):                                                    # GPT에게 질문하는 함수
    response = openai.ChatCompletion.create(model=model, messages=prompt)      # 오픈AI gpt의 모델과 메세지 전달
    system_message = response["choices"][0]["message"]                         # 시스템 메세지는 메세지 내용으로 리턴
    return system_message["content"]                                           # 응답받기 

In [ ]:
# 스트림릿 기준 TTS생성,재생하기
def TTS(response):                                                             # TTS를 활용하여 음성 파일 생성하기(자동으로 재생하기)
    # gTTS 를 활용하여 음성 파일 생성
    filename = "output.mp3"                                                    # 음성파일 지정하기.
    tts = gTTS(text=response,lang="ko")                                        # gtts로 음성화 시키기
    tts.save(filename)                                                         # tts를 저장하기
    # 생성은 여기까지 
    
    # 음원 파일 자동 재성 (사이트에서 재생바 만들어서 보이기)
    with open(filename, "rb") as f:                                            # 움원파일 열기 
        data = f.read()                                                        # 음원파일 읽어들이기
        b64 = base64.b64encode(data).decode()                                  # 음원파일 재생 패키지로 디코드 
        md = f"""
            <audio autoplay="True">
            <source src="data:audio/mp3;base64,{b64}" type="audio/mp3">
            </audio>
            """
        st.markdown(md,unsafe_allow_html=True,)                                # 재생바 생성하여 재생시키기
    # 파일 삭제
    os.remove(filename)                                                        # 사용 이후 파일삭제

In [ ]:
# 데이터 학습요령(사이트)
# https://y-rok.github.io/pytorch/2020/07/10/pytorch-dynamic-training.html
# https://pytorch.org/docs/stable/data.html#map-style-datasets

In [1]:
# # 파이프라인 연결
# from transformers import pipeline

# pipe = pipeline("text-generation", model="beomi/llama-2-ko-7b")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/llama-2-ko-7b")
model = AutoModelForCausalLM.from_pretrained("beomi/llama-2-ko-7b")